In [1]:
graphOutputs = [2,2,2,2,1,2,2,2] -- \lambda
graphStates = [1,2,3,4,5,6,7,8] -- \delta
graphTransitions = [[1,6,1,7,1,8,3,1], [4,5,2,6,3,4,1,4]] -- z_1, z_2

In [2]:
import Data.Function (on)
import Data.List (sortOn, groupBy, findIndex, transpose)
import Data.Maybe (fromJust)

type State = Int
type Transitions = [Int] -- in response to each input signal (length == number of signals)

regroupStates :: [[(State, Transitions)]] -> [[State]]
regroupStates = (regroup =<<)
    where
        regroup = removeTransitions . groupBy ((==) `on` transition) . sortOn transition
        transition = snd
        removeTransitions = ((fst <$>) <$>)

-- [[3], [1, 2]] -> [[1, 1], [1, 0], [0, 1]] -> [[(3, [1, 1])], [(1, [1, 0]), (2, [0, 1])]]
groupStatesTransitions :: [[State]] -> [Transitions] -> [[(State, Transitions)]]
groupStatesTransitions ((st : g) : gs) (t : ts) =
    let grp : rest = groupStatesTransitions (g : gs) ts in ((st, t) : grp) : rest
groupStatesTransitions [] _ = []
groupStatesTransitions ([] : gs) ts = [] : groupStatesTransitions gs ts

mapStateTransitions :: [[State]] -> [Transitions]
mapStateTransitions groups = stateTransitions <$> concat groups
    where
        stateTransitions st = (\ts -> groupWithStateIndex $ ts !! (st - 1)) <$> graphTransitions
        groupWithStateIndex st = (+ 1) $ fromJust $ findIndex (st `elem`) groups

In [3]:
import Data.List (intercalate)
import Control.Monad (when)
import Control.Monad.Writer

formatStateTransition :: Char -> [[(State, Transitions)]] -> String
formatStateTransition setch stts = columns $
    tableHead : statesRow : transitionRows ++ [tableTail]
    where
        tableHead = "\\begin{tabular}{|*{" ++ show (length states + 1) ++ "}{c|}}\n\\hline\n" ++ groupsRow
        tableTail = "\\end{tabular}"
        groupsRow = row ("     " : (groupItem <$> zip [1..] stts))
        groupItem (i, grp)
            | length grp == 1 = setName i
            | otherwise = "\\multicolumn{" ++ show (length grp) ++ "}{c|}{" ++ setName i ++ "}"
        statesRow = row ("     " : (stateName <$> states))
        transitionRows = row . transitionRow <$> zip [1..] transitions
        transitionRow (i, ts) = ("$z_" ++ show i ++ "$") : (setName <$> ts)
        setName i = '$' : setch : '_' : show i ++ "$"
        stateName i = "$a_" ++ show i ++ "$"
        row = intercalate " & "
        columns = intercalate "\\\\\\hline\n"
        states = fst <$> concat stts
        transitions = transpose $ snd <$> concat stts

formatGroups :: Char -> [[State]] -> String
formatGroups setch gs = '$' : (intercalate ", " $ fmt <$> (zip [1..] gs)) ++ "$"
    where
        fmt (grp, sts) = setch : '_' : show grp ++ " = \\{" ++ printSet sts ++ "\\}"
        printSet = intercalate ", " . (printState <$>)
        printState s = "a_" ++ show s

solve :: [[Int]] -> Char -> Writer String ()
solve gs set = do
    let ts = mapStateTransitions gs
    let stts = groupStatesTransitions gs ts
    let nextGs = regroupStates stts
    tell $ formatGroups set gs
    tell $ "\n"
    tell $ formatStateTransition set stts
    tell $ "\n"
    when (nextGs /= gs) $ solve nextGs (succ set)

let initGroups = regroupStates $ groupStatesTransitions [graphStates] (transpose [graphOutputs])
putStrLn $ snd $ runWriter $ solve initGroups 'B'

$B_1 = \{a_5\}, B_2 = \{a_1, a_2, a_3, a_4, a_6, a_7, a_8\}$
\begin{tabular}{|*{9}{c|}}
\hline
      & $B_1$ & \multicolumn{7}{c|}{$B_2$}\\\hline
      & $a_5$ & $a_1$ & $a_2$ & $a_3$ & $a_4$ & $a_6$ & $a_7$ & $a_8$\\\hline
$z_1$ & $B_2$ & $B_2$ & $B_2$ & $B_2$ & $B_2$ & $B_2$ & $B_2$ & $B_2$\\\hline
$z_2$ & $B_2$ & $B_2$ & $B_1$ & $B_2$ & $B_2$ & $B_2$ & $B_2$ & $B_2$\\\hline
\end{tabular}
$C_1 = \{a_5\}, C_2 = \{a_2\}, C_3 = \{a_1, a_3, a_4, a_6, a_7, a_8\}$
\begin{tabular}{|*{9}{c|}}
\hline
      & $C_1$ & $C_2$ & \multicolumn{6}{c|}{$C_3$}\\\hline
      & $a_5$ & $a_2$ & $a_1$ & $a_3$ & $a_4$ & $a_6$ & $a_7$ & $a_8$\\\hline
$z_1$ & $C_3$ & $C_3$ & $C_3$ & $C_3$ & $C_3$ & $C_3$ & $C_3$ & $C_3$\\\hline
$z_2$ & $C_3$ & $C_1$ & $C_3$ & $C_2$ & $C_3$ & $C_3$ & $C_3$ & $C_3$\\\hline
\end{tabular}
$D_1 = \{a_5\}, D_2 = \{a_2\}, D_3 = \{a_3\}, D_4 = \{a_1, a_4, a_6, a_7, a_8\}$
\begin{tabular}{|*{9}{c|}}
\hline
      & $D_1$ & $D_2$ & $D_3$ & \multicolumn{5}{c|}{$D_4$}\\\hline
      & $a_5